In [1]:
#Import Bibliotecas
import pypsa
import pandas as pd
import numpy as np


In [2]:
# =====================
# Grundparameter Standort
# =====================

roof_area = 5500                 # Gebäudefläche [m²]
number_transporter = 78          # Anzahl E-Fahrzeuge
capacitiy_battery_transporter = 40  # Batteriekapazität je Fahrzeug [kWh]
inhouse_pv = 0.35                # Max. PV-Leistung auf dem Dach [MW]

# ====================================
# Wärmebedarfe (vereinfachte Annahmen)
# ====================================

DHW_load_hourly = roof_area * 0.008 / 8760      # Trinkwarmwasser 
Thermal_load_hourly = roof_area * 0.05 / 8760   # Raumheizung 

# =========================================
#Elektrische Lasten (vereinfachte Annahmen)
# =========================================

building_load_hourly = roof_area*0.03 /8760 # 
fleet_load_hourly = number_transporter * capacitiy_battery_transporter / 24 / 1000 # 


# ===========
# Zeitprofile
# ===========

# Lade das ninja_pv Solar-Profil für Deutschland (51.16°N, 13°E)
renewable_profile_solar = (pd.read_csv("ninja_pv_51.1601_13.csv", usecols=["electricity"]) ["electricity"].astype(float))  # Stündliches, normiertes PV-Erzeugungsprofil

custom_load_profile_MW = (pd.read_csv("jahreslastgang_dhl (1).csv", usecols=["load_kWh"])
    ["load_kWh"].astype(float) / 1000) # Standortlastgang, von kWh/h in MW umgerechnet

In [3]:
# ===============================
#netzwerk erstellen with Batterie
# ===============================

with_battery = pypsa.Network()
with_battery.set_snapshots(range(8760))

#hinzufügen bus
with_battery.add("Bus", name="electricity bus", carrier="electricity")
with_battery.add("Bus", name= "h_bus", carrier="heat")
with_battery.add("Bus", name= "h_bus2", carrier="heat")

#hinzufügen generatoren
with_battery.add("Generator", name="PV", bus="electricity bus", p_nom=inhouse_pv, p_max_pu=renewable_profile_solar, marginal_cost=1)  # PV Betriebskosten: 1 €/MWh
with_battery.add("Generator", name="Grid", bus="electricity bus", p_nom_extendable=True, marginal_cost=150)  # Netzbezug: 150 €/MWh (15 Cent/kWh), fixe Leistung 10 MW

#hinzufügen last
with_battery.add("Load", name="building load", bus="electricity bus", p_set=custom_load_profile_MW*0.15)  # *0.15 da 15% des Lastprofils auf Gebäude entfallen
with_battery.add("Load", name="fleet load", bus="electricity bus", p_set=custom_load_profile_MW*0.7)  #
with_battery.add("Load", "DHW_Load",bus="h_bus", p_set=custom_load_profile_MW * 0.05)
with_battery.add("Load", "Thermal_Load",bus="h_bus2", p_set=custom_load_profile_MW * 0.1)

#hinzufügen link
with_battery.add("Link", name="h_pump", bus0="electricity bus", bus1="h_bus", efficiency=3.5, p_nom_extendable=True)
with_battery.add("Link", name="h_pump2", bus0="electricity bus", bus1="h_bus2", efficiency=3.5, p_nom_extendable=True)

#hinzufügen Wärmespeicher
with_battery.add("Store", name="DHW_Storage", bus="h_bus", carrier="heat", e_nom_extendable=True, e_cyclic=True, capital_cost=1)  # 5 MWh Speicher, 0.5% Verlust/h
with_battery.add("Store", name="Thermal_Storage", bus="h_bus2", carrier="heat", e_nom_extendable=True, e_cyclic=True, capital_cost=1)

#hinzufügen Batteriespeicher
with_battery.add("Store", name="Battery", bus="electricity bus", carrier="electricity", e_nom_extendable=True, e_cyclic=True, capital_cost=1)

In [4]:
with_battery.optimize(solver_name='highs')

Index(['electricity bus', 'h_bus', 'h_bus2'], dtype='object', name='name')
Index(['h_pump', 'h_pump2'], dtype='object', name='name')
Index(['DHW_Storage', 'Thermal_Storage', 'Battery'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 5/5 [00:00<00:00, 304.12it/s]
INFO:linopy.io: Writing time: 0.34s
Status: warning
Termination condition: infeasible
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: Infeasible



Running HiGHS 1.12.0 (git hash: n/a): Copyright (c) 2025 HiGHS under MIT licence terms
LP linopy-problem-rvgnv7wk has 175202 rows; 87602 cols; 297842 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 4e+00]
  Cost    [5e+00, 5e+00]
  Bound   [0e+00, 0e+00]
  RHS     [3e-09, 1e+00]
Presolving model
35040 rows, 56670 cols, 109230 nonzeros  0s
26280 rows, 47910 cols, 91710 nonzeros  0s
Problem status detected on presolve: Infeasible
Model name          : linopy-problem-rvgnv7wk
Model status        : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.12


('warning', 'infeasible')

In [ ]:
# ============================
# Ergebnisse abrufen & Analyse
# ============================

# 1. Optimale Kapazitäten (extendable components)
print("=== Optimale Kapazitäten ===")
print(f"Grid Leistung: {with_battery.generators.loc['Grid', 'p_nom_opt']:.2f} MW")
print(f"Grid Leistung: {with_battery.generators.loc['PV', 'p_nom_opt']:.2f} MW")
print(f"Wärmepumpe 1: {with_battery.links.loc['h_pump', 'p_nom_opt']:.2f} MW")
print(f"Wärmepumpe 2: {with_battery.links.loc['h_pump2', 'p_nom_opt']:.2f} MW")
print(f"DHW Speicher: {with_battery.stores.loc['DHW_Storage', 'e_nom_opt']:.2f} MWh")
print(f"Thermal Speicher: {with_battery.stores.loc['Thermal_Storage', 'e_nom_opt']:.2f} MWh")
print(f"Battery: {with_battery.stores.loc['Battery', 'e_nom_opt']:.2f} MWh")

# 2. Energiemengen pro Komponente
print("\n=== Energiemengen (MWh/Jahr) ===")
pv_energy = with_battery.generators_t.p["PV"].sum()
grid_energy = with_battery.generators_t.p["Grid"].sum()
print(f"PV Erzeugung: {pv_energy:.1f} MWh")
print(f"Netzbezug: {grid_energy:.1f} MWh")

# 3. Kosten-Aufschlüsselung
print("\n=== Kosten-Aufschlüsselung (€/Jahr) ===")
pv_cost = pv_energy * 1  # marginal_cost=1
grid_cost = grid_energy * 150  # marginal_cost=150
h_pump_energy = with_battery.links_t.p0["h_pump"].sum() + with_battery.links_t.p0["h_pump2"].sum()
h_pump_cost = h_pump_energy * 2  # marginal_cost=2
print(f"PV Kosten: {pv_cost:.0f} €")
print(f"Grid Kosten: {grid_cost:.0f} €")
print(f"Wärmepumpen Betrieb: {h_pump_cost:.0f} €")
print(f"Gesamtkosten (Betrieb): {pv_cost + grid_cost + h_pump_cost:.0f} €")

# 4. Optimale Zielfunktion
print(f"\n=== Gesamtoptimierungskosten: {with_battery.objective:.0f} € ===")

# 5. Last-Analyse
print("\n=== Lasten (MWh/Jahr) ===")
building_load = with_battery.loads_t.p["building load"].sum()
fleet_load = with_battery.loads_t.p["fleet load"].sum()
dhw_load = with_battery.loads_t.p["DHW_Load"].sum()
thermal_load = with_battery.loads_t.p["Thermal_Load"].sum()
print(f"Gebäudelast: {building_load:.1f} MWh")
print(f"Fahrzeuglast: {fleet_load:.1f} MWh")
print(f"DHW-Last: {dhw_load:.1f} MWh")
print(f"Thermische Last: {thermal_load:.1f} MWh")
print(f"Gesamt-Last: {building_load + fleet_load + dhw_load + thermal_load:.1f} MWh")

=== Optimale Kapazitäten ===
Grid Leistung: 5.78 MW
Wärmepumpe 1: 0.01 MW
Wärmepumpe 2: 0.01 MW
DHW Speicher: -0.00 MWh
Thermal Speicher: -0.00 MWh
Battery: 8.14 MWh

=== Energiemengen (MWh/Jahr) ===
PV Erzeugung: 458.1 MWh
Netzbezug: 466.0 MWh

=== Kosten-Aufschlüsselung (€/Jahr) ===
PV Kosten: 458 €
Grid Kosten: 69897 €
Wärmepumpen Betrieb: 89 €
Gesamtkosten (Betrieb): 70443 €

=== Gesamtoptimierungskosten: 70363 € ===

=== Lasten (MWh/Jahr) ===
Gebäudelast: 155.2 MWh
Fahrzeuglast: 724.5 MWh
DHW-Last: 51.7 MWh
Thermische Last: 103.5 MWh
Gesamt-Last: 1035.0 MWh
